In [1]:
import numpy as np 
import pandas as pd
import requests
from tqdm import tqdm
from time import sleep
import ast, json, re
from glob import glob
from bs4 import BeautifulSoup
import warnings, wget
warnings.filterwarnings('ignore')
data_folder = '' #add path to data folder here
api = '' #add UMLS API key here

## Download files from UMLS
https://www.nlm.nih.gov/research/umls/licensedcontent/umlsknowledgesources.html

### Download

In [ ]:
url = "https://uts-ws.nlm.nih.gov/download?url=https://download.nlm.nih.gov/umls/kss/2022AB/umls-2022AB-full.zip&apiKey="+api
out = data_folder+'external_data'
wget.download(url, out=out)

'/content/drive/MyDrive/008_JADER/pmdacasereport202302/kegg_drug_labels/external_data/umls-2022AB-full.zip'

In [ ]:
import zipfile
with zipfile.ZipFile(out+'/umls-2022AB-full.zip', 'r') as zip_ref:
    zip_ref.extractall(out)

In [ ]:
url = "https://uts-ws.nlm.nih.gov/download?url=https://download.nlm.nih.gov/umls/kss/2022AB/umls-2022AB-mrconso.zip&apiKey="+api
out = data_folder+'external_data'
wget.download(url, out=out)

In [ ]:
import zipfile
with zipfile.ZipFile(out+'/umls-2022AB-mrconso.zip', 'r') as zip_ref:
    zip_ref.extractall(out)

### Obtain only JP MedDRA
https://www.ncbi.nlm.nih.gov/books/NBK9685/table/ch03.T.concept_names_and_sources_file_mr/?report=objectonly


In [ ]:
jp_meddra = []
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'MDRJPN':
      jp_meddra.append(line.split('|')) 

17213334it [00:34, 504691.70it/s]


In [ ]:
jp_meddra_df = pd.DataFrame(jp_meddra, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
jp_meddra_df = jp_meddra_df.drop([''], axis = 1)
print(jp_meddra_df.shape)
jp_meddra_df.to_csv(out+'umls_meddra_jp.csv', index=False)
jp_meddra_df.head()

(216817, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000727,JPN,P,L3471380,PF,S3998874,N,A11418933,,,10000647,MDRJPN,PT,10000647,急性腹症,3,N,
1,C0000727,JPN,P,L3471380,PF,S3998874,N,A26215806,,,10000647,MDRJPN,LLT,10000647,急性腹症,3,N,
2,C0000727,JPN,S,L6278809,PF,S7192299,N,A12736025,,,10000647,MDRJPN,OLJKN,10042784,ｷｭｳｾｲﾌｸﾌﾞｼｮｳｺｳｸﾞﾝ,3,O,
3,C0000727,JPN,S,L6278809,PF,S7192299,Y,A26336094,,,10000647,MDRJPN,LLTJKN,10000096,ｷｭｳｾｲﾌｸﾌﾞｼｮｳｺｳｸﾞﾝ,3,N,
4,C0000727,JPN,S,L6290235,PF,S7192292,N,A26305977,,,10000647,MDRJPN,LLTJKN,10000647,ｷｭｳｾｲﾌｸｼｮｳ,3,N,


### Obtain only EN MedDRA

In [ ]:
en_meddra = []
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'MDR':
      en_meddra.append(line.split('|')) 

17213334it [00:32, 528050.44it/s]


In [ ]:
en_meddra_df = pd.DataFrame(en_meddra, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
en_meddra_df = en_meddra_df.drop([''], axis = 1)
print(en_meddra_df.shape)
en_meddra_df.to_csv(out+'umls_meddra_en.csv', index=False)
en_meddra_df.head()

(116807, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639292,,,10000647,MDR,PT,10000647,Acute abdomen,3,N,256
1,C0000727,ENG,P,L0000727,VCW,S0584932,N,A25741630,,,10000647,MDR,LLT,10000647,Acute abdomen,3,N,256
2,C0000727,ENG,S,L0161339,PF,S1616740,Y,A25720821,,,10000647,MDR,LLT,10042784,Syndrome abdominal acute,3,N,
3,C0000727,ENG,S,L0161339,VCW,S1616739,Y,A25708511,,,10000647,MDR,LLT,10000096,Abdominal syndrome acute,3,N,
4,C0000729,ENG,P,L0000729,VC,S0353650,N,A25716812,,,10000081,MDR,LLT,10000057,Abdominal cramps,3,N,256


### Make RxNORM Table

In [ ]:
rxnorm = []
out = data_folder+'external_data'
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'RXNORM':
      rxnorm.append(line.split('|')) 

In [ ]:
rxnorm_umls = pd.DataFrame(rxnorm, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
rxnorm_umls = rxnorm_umls.drop([''], axis = 1)
print(rxnorm_umls.shape)
out = data_folder+'external_data/'
rxnorm_umls.to_csv(out+'umls_rxnorm.csv', index=False)
rxnorm_umls.head()

(322769, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921,1926948,,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256
1,C0000266,ENG,P,L0030574,PF,S0071080,Y,A10336090,829,38,,RXNORM,BN,38,Parlodel,0,N,4352
2,C0000294,ENG,P,L0025479,PF,S0525354,N,A31762929,12251526,44,,RXNORM,IN,44,mesna,0,N,4352
3,C0000378,ENG,P,L0565039,PF,S14426412,N,A31685600,12254325,1489913,,RXNORM,IN,1489913,droxidopa,0,N,4352
4,C0000392,ENG,P,L0005188,PF,S11855837,N,A31642632,12254378,61,,RXNORM,IN,61,beta-alanine,0,N,4352


### Filter for SNOMED

In [ ]:
en_snomed = []
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'SNOMEDCT_US':
      en_snomed.append(line.split('|')) 

17213334it [00:57, 300827.92it/s]


In [ ]:
snomed_umls = pd.DataFrame(en_snomed, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
snomed_umls = snomed_umls.drop([''], axis = 1)
print(snomed_umls.shape)
out = data_folder+'external_data/'
snomed_umls.to_csv(out+'umls_snomed.csv', index=False)
snomed_umls.head()

(1597441, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000039,ENG,S,L0012507,PF,S0033298,N,A22817493,166113012,102735002,,SNOMEDCT_US,OAP,102735002,Dipalmitoylphosphatidylcholine,9,O,256
1,C0000039,ENG,S,L3000054,PF,S3260062,Y,A22880204,544223010,102735002,,SNOMEDCT_US,OAF,102735002,Dipalmitoylphosphatidylcholine (substance),9,O,
2,C0000052,ENG,P,L0000052,VC,S0575717,Y,A27769867,97197014,58488005,,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",9,N,
3,C0000052,ENG,S,L0006129,VC,S0604824,Y,A27781005,97198016,58488005,,SNOMEDCT_US,SY,58488005,Branching enzyme,9,N,256
4,C0000052,ENG,S,L0455267,PF,S0589116,Y,A27766654,97199012,58488005,,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",9,N,


### Filter for MSHJPN

In [ ]:
jp_msh = []
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'MSHJPN':
      jp_msh.append(line.split('|')) 

17213334it [00:41, 411399.98it/s]


In [ ]:
mshjpn = pd.DataFrame(jp_msh, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
mshjpn = mshjpn.drop([''], axis = 1)
print(mshjpn.shape)
out = data_folder+'external_data/'
mshjpn.to_csv(out+'umls_mshjpn.csv', index=False)
mshjpn.head()

(124075, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000039,JPN,P,L3437320,PF,S3964814,Y,A15722927,,M0023172,D015060,MSHJPN,PT,D015060,"1,2-ジパルミトイルホスファチジルコリン",3,N,
1,C0000039,JPN,S,L3437321,PF,S3964815,Y,A7887644,,M0023172,D015060,MSHJPN,SY,D015060,"1,2-ジパルミトイルレシチン",3,N,
2,C0000039,JPN,S,L3449844,PF,S3977338,Y,A7895635,,M0023172,D015060,MSHJPN,SY,D015060,ジパルミトイルホスファチジルコリン,3,N,
3,C0000039,JPN,S,L3449845,PF,S3977339,Y,A7895636,,M0023172,D015060,MSHJPN,SY,D015060,ジパルミトイルレシチン,3,N,
4,C0000052,JPN,P,L3437333,PF,S3964827,Y,A24691790,,M0023173,D015061,MSHJPN,PT,D015061,"1,4-アルファ-グルカン分枝酵素",3,N,


### Filter for ICD10

In [5]:
icd = []
out = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/external_data'
with open(out+'/MRCONSO.RRF') as f:
  for line in tqdm(f):
    if line.split('|')[11] == 'ICD10' or line.split('|')[11] == 'ICD10CM':
      icd.append(line.split('|')) 

17213334it [00:56, 304608.13it/s]


In [7]:
icd_df = pd.DataFrame(icd, columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF',
                                                  'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE',
                                                  'STR', 'SRL', 'SUPPRESS', 'CVF', ''])
icd_df = icd_df.drop([''], axis = 1)
print(icd_df.shape)
icd_df.to_csv(out+'/umls_icd.csv', index=False)
icd_df.head()

(220399, 18)


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639289,,,,ICD10,PT,R10.0,Acute abdomen,3,N,256
1,C0000727,ENG,P,L0000727,VCW,S0584932,N,A17803269,,,R10.0,ICD10CM,PT,R10.0,Acute abdomen,4,N,256
2,C0000727,ENG,P,L0000727,VCW,S0584932,N,A20117915,,,R10.0,ICD10CM,AB,R10.0,Acute abdomen,4,Y,256
3,C0000737,ENG,S,L9385397,PF,S11687982,N,A20162941,,,R10.9,ICD10CM,AB,R10.9,Unspecified abdominal pain,4,Y,256
4,C0000737,ENG,S,L9385397,PF,S11687982,Y,A17867148,,,R10.9,ICD10CM,PT,R10.9,Unspecified abdominal pain,4,N,256


---
## Map ATHENA - MedDRA data to UMLS

In [ ]:
athena = data_folder+'external_data/athena_meddra/'
df = pd.read_csv(athena+'CONCEPT.csv', error_bad_lines = False, delimiter = '\t')
df = df[df.vocabulary_id == 'MedDRA']
df.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1797,36044927,Ventilation pneumonitis,Condition,MedDRA,LLT,NaN,10000001,19700101,20160311,D
1798,35305836,11-beta-hydroxylase deficiency,Condition,MedDRA,PT,C,10000002,19700101,20991231,NaN
1799,36357176,11-oxysteroid activity incr,Condition,MedDRA,LLT,NaN,10000003,19700101,20130613,D
1800,36357177,11-oxysteroid activity increased,Measurement,MedDRA,LLT,C,10000004,19700101,20991231,NaN
1801,36312355,17 ketosteroids urine,Measurement,MedDRA,PT,C,10000005,19700101,20991231,NaN


In [ ]:
concept_codes = df.concept_code.unique().tolist()
maps = []
for code in tqdm(concept_codes):
  url = 'https://uts-ws.nlm.nih.gov/rest/content/current/source/MDRJPN/{c}/atoms?apiKey={a}'.format(a = api, c = str(code))
  r = requests.get(url).json()
  try:
    results = r['result']
    maps.append([code, results])
  except:
    maps.append([code, None])

100%|██████████| 108301/108301 [9:32:50<00:00,  3.15it/s]


In [ ]:
athena_map = pd.DataFrame(maps, columns = ['code', 'umls_results'])
athena_map.to_csv(athena+'umls_map.csv', index=False)

---

In [ ]:
athena = data_folder+'external_data/athena_meddra/'
athena_map = pd.read_csv(athena+'umls_map.csv')
athena_map.head(1)

,code,umls_results
0,10000001,NaN


In [ ]:
mapped = athena_map[athena_map.umls_results.astype(str) != 'nan']
mapped['umls_results'] = mapped.umls_results.apply(lambda x: ast.literal_eval(x) if str(x) != 'nan' else None)
mapped['umls_code'] = mapped.umls_results.apply(lambda x: list([i['concept'].split('/')[-1] for i in x]) if x != None else None)
mapped['umls_termtype'] = mapped.umls_results.apply(lambda x: [i['termType'] for i in x] if x!= None else None)
mapped['jp_name'] = mapped.umls_results.apply(lambda x: [i['name'] for i in x] if x!=None else None)
mapped = mapped[['code', 'umls_code', 'umls_termtype', 'jp_name']].explode(['umls_code', 'umls_termtype', 'jp_name'])
mapped.head(1)

,code,umls_code,umls_termtype,jp_name
1,10000002,C0268292,LLT,１１βヒドロキシラーゼ欠損症


In [ ]:
mapped.to_csv(athena+'meddra_jp.csv', index=False)

In [ ]:
nan = athena_map[athena_map.umls_results.astype(str) == 'nan'].code.tolist()

In [ ]:
unmapped_concepts = df[df.concept_code.astype(int).isin(nan)]
print(unmapped_concepts.shape) #of 44,872 mapped, 21,033 were LLT duplicate of PTs. 
unmapped_concepts.head(1)

(44872, 10)


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1797,36044927,Ventilation pneumonitis,Condition,MedDRA,LLT,NaN,10000001,19700101,20160311,D


In [ ]:
athena_map['umls_results'] = athena_map.umls_results.apply(lambda x: ast.literal_eval(x) if str(x) != 'nan' else None)
athena_map['umls_code'] = athena_map.umls_results.apply(lambda x: list([i['concept'].split('/')[-1] for i in x]) if x != None else None)
athena_map['umls_termtype'] = athena_map.umls_results.apply(lambda x: [i['termType'] for i in x] if x!= None else None)
athena_map['jp_name'] = athena_map.umls_results.apply(lambda x: [i['name'] for i in x] if x!=None else None)
athena_map.head(1)

,code,umls_results,umls_code,umls_termtype,jp_name
0,10000001,None,None,None,None
